In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as db
import psycopg2
import pandas as pd
import time
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Set your Spotify API credentials - Vigs
client_id = 'e6d6060806824f65b988d05e2c6a5a74'
client_secret = '3aa225c62c1542a6b1dce64fa69d18d4'

# Initialize Spotify client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

In [3]:
# Connect to DB in elephant SQL
db_params = {
    "host": "rain.db.elephantsql.com",
    "dbname": "auspovuc", 
    "user": "auspovuc",  
    "password": "bmJdG19Daw9rkEsJ3VnkefRGCBF_oy7F",  
}

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [9]:
#View creation walkthrough, Do not run yet : (Add if exist statement)

start = time.process_time()

# Declare conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

# Declare age_group
groups = ['early_years', 'young_adults', 'mature_adults', 'middle_age', 'elderly']

# Create Views for each condition and age_group, ranking genres
for i in range(len(conditions)):
    for n in range(len(groups)):
        query = f"""
            CREATE OR REPLACE VIEW {groups[n]}_{conditions[i]}_genre AS(
                SELECT a.genre, COUNT(*),
                  ROW_NUMBER() OVER(ORDER BY COUNT(*) DESC) AS row_n
                FROM (
                  SELECT s.respondent, g.*
                  FROM survey s                      
                  CROSS JOIN LATERAL (
                     values (s.freq1, 'freq1'),
                            (s.freq2, 'freq2'),
                            (s.freq3, 'freq3')
                        ) as g(genre, frequency)
                WHERE s.{conditions[i]} >= 7
                AND age_group = '{groups[n]}') AS a
                WHERE a.genre IS NOT NULL
                GROUP BY a.genre
                )
                """
        cur.execute(query,)
        conn.commit()

print(time.process_time() - start)

0.0


In [10]:
# Ask user's age_group
age = input('Please indicate your age group:\n  1.0-17 \n  2.18-34 \n  3.35-59 \n  4.60-74 \n  5.76 and above\nEnter a number: ')

Please indicate your age group:
  1.0-17 
  2.18-34 
  3.35-59 
  4.60-74 
  5.76 and above
Enter a number: 4


In [11]:
# Ask if user has any of the 4 conditions
feels = input('How are you feeling today (select a number)?\n  1.Anxious \n  2.Depressed \n  3.Can\'t sleep \n  4.I have OCD \n  5.I feel great!\nEnter a number: ')

How are you feeling today (select a number)?
  1.Anxious 
  2.Depressed 
  3.Can't sleep 
  4.I have OCD 
  5.I feel great!
Enter a number: 3


In [12]:
# Define conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

# Define age_group
groups = ['early_years', 'young_adults', 'mature_adults', 'middle_age', 'elderly']

# Ensure print dataframe with full column width for url display without truncation
pd.set_option('display.max_colwidth', None)  

# look for top choice of genre based on user input (*SQL INJECTION RISK)
if feels in ['1', '2', '3', '4']:
    if age in ['1', '2', '3', '4', '5']:
        x = int(feels)
        y = int(age)
        # query recommended_tracks for genre from specified view
        cur.execute(f"""
                SELECT t.track_name, t.artist, t.album, t.genre, t.preview_url 
                FROM recommended_tracks t
                JOIN {groups[y-1]}_{conditions[x-1]}_genre a
                ON a.genre = t.genre
                WHERE a.row_n = 1 ;
                """)
        conn.commit()
        result = cur.fetchall()
        #convert result to a dataframe and print    
        df = pd.DataFrame(result, columns=['Song', 'Artist', 'Album', 'Genre', 'Preview URL'])
        print('Have a listen to these songs. They just might make you feel better!\n\n', df)
    else:
        print('Hmm, you\'re good, you don\'t need help.')
    
elif feels == '5':
    # Define seed genres
    seed_genres = ['pagode']
    # Get recommended tracks for each seed genre
    for seed_genre in seed_genres:
        recommended_tracks = sp.recommendations(seed_genres=[seed_genre], limit=3, country="SG")
        for track in recommended_tracks["tracks"]:
            track_data = {
                        "track_name": track["name"],
                        "artist": ", ".join(artist["name"] for artist in track["artists"]),
                        "album": track["album"]["name"],
                        "genre": seed_genre,
                        "preview_url": track["preview_url"]
                            }
        # Create a DataFrame from the track data, suggest random track from top 3
        random_track = {random.randint(0,3)}
        df = pd.DataFrame(track_data, index=[random_track])
        print('Check this out!\n', df)
else:
    print('Hmm, you\'re good, you don\'t need help.')

Have a listen to these songs. They just might make you feel better!

                                                   Song  \
0                                Body Like A Back Road   
1  WDYW (feat. Lil Uzi Vert, A$AP Ferg & Rich The Kid)   
2                That's What I Like (feat. Gucci Mane)   

                                           Artist  \
0                                        Sam Hunt   
1  Carnage, Lil Uzi Vert, A$AP Ferg, Rich The Kid   
2                          Bruno Mars, Gucci Mane   

                                           Album    Genre  \
0                          Body Like A Back Road  hip-hop   
1                                     Papi Gordo  hip-hop   
2  That's What I Like (feat. Gucci Mane) [Remix]  hip-hop   

                                                                                                   Preview URL  
0                                                                                                         None  
1  https://p.

In [ ]:
# close connection to db
conn.close()